In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame,read_csv
from sklearn.utils import shuffle

In [2]:
df = read_csv('heart_failure.csv')
df.head(5)


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [57]:
df['anaemia'] = np.where(df['anaemia'] == 1 ,True,False)
df['diabetes'] = np.where(df['diabetes'] == 1, True, False)
df['high_blood_pressure'] = np.where(df['high_blood_pressure'] == 1, True, False)
df['smoking'] = np.where(df['smoking'] == 1, True, False)
df['sex'] = np.where(df['sex'] == 1, 'Male','Female')


In [4]:
#cpk normal values from 10 to 120 micrograms per liter
def set_cpk(row):
    if row["creatinine_phosphokinase"] >=10 and row["creatinine_phosphokinase"] <= 120:
        return "Normal"
    else:
        return "High"
    
df = df.assign(cp_desc = df.apply(set_cpk, axis = 1))


In [5]:
def set_eject_fract(row):
    if row["ejection_fraction"] <= 35:
        return "Low"
    elif row["ejection_fraction"] > 35 and row["ejection_fraction"] <= 49:
        return "Below_Normal"
    elif row["ejection_fraction"] > 50 and row["ejection_fraction"] <= 75:
        return "Normal"
    else:
        return "High"
df['ejection_fraction_desc'] =  df.apply(set_eject_fract, axis =1)

In [6]:
def set_platelets(row):    
    if row["sex"] == 'Female':  #females
        if row["platelets"] < 157000:
            return "Low"
        elif row["platelets"] >=157000 and row["platelets"] <= 371000:
            return "Normal"
        else:
            return "High"
        
    elif row["sex"] == 'Male':  #males
        if row["platelets"] < 135000:
            return "Low"
        if row["platelets"] >= 135000 and row["platelets"] <= 317000:
            return "Normal"
        else:
            return "High"
df['platelets_desc'] = df.apply(set_platelets, axis = 1)

In [7]:
def set_sodium(row):
    if row["serum_sodium"] < 135:
        return "Low"
    elif row["serum_sodium"] >=135 and row["serum_sodium"] <= 145:
        return "Normal"
    else:
        return "High"
df['sodium_desc'] = df.apply(set_sodium, axis =1)

In [8]:
def set_creatinine(row):
    if row["sex"] == 'Female':  #females
        if row["serum_creatinine"] >=0.5 and row["serum_creatinine"] <= 1.1:
            return "Normal"
        else:
            return "High"
        
    elif row["sex"] == 'Male':  #males
        if row["serum_creatinine"] >=0.6 and row["serum_creatinine"] <= 1.2:
            return "Normal"
        else:
            return "High"
df['serum_creatinine_desc'] = df.apply(set_creatinine, axis = 1)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    int64  
 1   anaemia                   299 non-null    bool   
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    bool   
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    bool   
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    object 
 10  smoking                   299 non-null    bool   
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
 13  cp_desc                   299 non-null    object 
 14  ejection_f

In [12]:
X = df.drop('DEATH_EVENT', axis = 1)
y = df['DEATH_EVENT']
print(X.shape)
X.head(2)

(299, 17)


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,cp_desc,ejection_fraction_desc,platelets_desc,sodium_desc,serum_creatinine_desc
0,75,False,582,False,20,True,265000.00,1.9,130,Male,False,4,High,Low,Normal,Low,High
1,55,False,7861,False,38,False,263358.03,1.1,136,Male,False,6,High,Below_Normal,Normal,Normal,Normal


In [13]:
import imblearn
from imblearn.over_sampling import SMOTENC
smote = SMOTENC(random_state=1,categorical_features=[0,1,3,5,9,10,12,13,14,15,16])
x_bal, y_bal = smote.fit_sample(X, y)
x_bal = DataFrame(x_bal, columns = X.columns)
print(Series(y_bal).value_counts())
x_bal['serum_creatinine_desc'].value_counts()

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/loca

1    203
0    203
dtype: int64


Normal    218
High      188
Name: serum_creatinine_desc, dtype: int64

In [14]:
encode = ['sex','cp_desc','ejection_fraction_desc','platelets_desc',
         'sodium_desc','serum_creatinine_desc']
x = pd.get_dummies(x_bal,columns = encode, drop_first = True)
print(x.shape)
x.head()

(406, 21)


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,smoking,time,sex_Male,cp_desc_Normal,ejection_fraction_desc_High,ejection_fraction_desc_Low,ejection_fraction_desc_Normal,platelets_desc_Low,platelets_desc_Normal,sodium_desc_Low,sodium_desc_Normal,serum_creatinine_desc_Normal
0,75,False,582,False,20,True,265000,1.9,130,False,4,1,0,0,1,0,0,1,1,0,0
1,55,False,7861,False,38,False,263358,1.1,136,False,6,1,0,0,0,0,0,1,0,1,1
2,65,False,146,False,20,False,162000,1.3,129,True,7,1,0,0,1,0,0,1,1,0,0
3,50,True,111,False,20,False,210000,1.9,137,False,7,1,1,0,1,0,0,1,0,1,0
4,65,True,160,True,20,False,327000,2.7,116,False,8,0,0,0,1,0,0,1,1,0,0


In [15]:
col = ['age','creatinine_phosphokinase','ejection_fraction',
       'platelets','serum_creatinine','serum_sodium','time',
       'anaemia','diabetes','high_blood_pressure','smoking',
       'sex_Male','cp_desc_Normal','ejection_fraction_desc_High',
       'ejection_fraction_desc_Low','ejection_fraction_desc_Normal',
       'platelets_desc_Low','platelets_desc_Normal','sodium_desc_Low',
       'sodium_desc_Normal','serum_creatinine_desc_Normal']

In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

col_trans = ColumnTransformer(remainder='passthrough',
                              transformers = [('scaler',MinMaxScaler(),[0,2,4,6,7,8,10])])
trans = col_trans.fit_transform(x)
trans = DataFrame(trans,columns = col)
trans.head()

,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,time,anaemia,diabetes,high_blood_pressure,smoking,sex_Male,cp_desc_Normal,ejection_fraction_desc_High,ejection_fraction_desc_Low,ejection_fraction_desc_Normal,platelets_desc_Low,platelets_desc_Normal,sodium_desc_Low,sodium_desc_Normal,serum_creatinine_desc_Normal
0,0.636364,0.0713192,0.0909091,0.290823,0.157303,0.485714,0,False,False,True,False,1,0,0,1,0,0,1,1,0,0
1,0.272727,1,0.363636,0.288833,0.0674157,0.657143,0.00711744,False,False,False,False,1,0,0,0,0,0,1,0,1,1
2,0.454545,0.0156928,0.0909091,0.16596,0.0898876,0.457143,0.0106762,False,False,False,True,1,0,0,1,0,0,1,1,0,0
3,0.181818,0.0112274,0.0909091,0.224148,0.157303,0.685714,0.0106762,True,False,False,False,1,1,0,1,0,0,1,0,1,0
4,0.454545,0.0174789,0.0909091,0.365984,0.247191,0.0857143,0.0142349,True,True,False,False,0,0,0,1,0,0,1,1,0,0


In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(trans,y_bal, test_size = 0.2, random_state = 0)

In [ ]:
#from sklearn.model_selection import GridSearchCV
#from sklearn.ensemble import RandomForestClassifier
#grid = GridSearchCV(RandomForestClassifier(),
 #               {'max_depth':range(1,15), 'n_estimators': [50, 100,300],
  #              'max_features': ['auto', 'sqrt', 'log'], 
   #             'min_samples_split': [2,3,5,7,9],'min_samples_leaf': [1,2,4,6,8]},
    #             cv = 3, n_jobs = 2, scoring = 'recall', verbose = 1)

#grid.fit(x_train,y_train)

In [ ]:
grid.best_params_

In [50]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_depth = 7, max_features= 'sqrt',random_state = 2,
                                    min_samples_leaf = 1,
                                    min_samples_split = 5,n_estimators = 100)
classifier.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=7, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=2, verbose=0,
                       warm_start=False)

In [51]:
y_pred = classifier.predict(x_test)

In [52]:
from sklearn.metrics import classification_report, recall_score, accuracy_score,precision_score, f1_score, confusion_matrix
recall = recall_score(y_test,y_pred)
accuracy = accuracy_score(y_test,y_pred)
precision = precision_score(y_test,y_pred)
f1score = f1_score(y_test,y_pred)
print('the Recall for tuned RandomForest is:{}'.format(round(recall,4)))
print(f'the Accuracy for tuned RandomForest is:{round(accuracy,4)}')
print('the Precision for tuned RandomForest is: %s' %(round(precision,4)))
print(f'the F1_score for tuned RandomForest is:{round(f1score,4)}')

the Recall for tuned RandomForest is:0.9787
the Accuracy for tuned RandomForest is:0.939
the Precision for tuned RandomForest is: 0.92
the F1_score for tuned RandomForest is:0.9485


In [53]:
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.89      0.93        35
           1       0.92      0.98      0.95        47

    accuracy                           0.94        82
   macro avg       0.94      0.93      0.94        82
weighted avg       0.94      0.94      0.94        82



In [54]:
matrix = confusion_matrix(y_test,y_pred)
print(matrix)

[[31  4]
 [ 1 46]]


In [56]:
import pickle
pickle.dump(classifier, open('heart_featured.pkl','wb'))